In [1]:
import pandas as pd
import cufflinks as cf
cf.go_offline()

In [2]:
area = pd.read_csv('DXYArea.csv')
area['updateTime'] = pd.to_datetime(area['updateTime'])

province_cols = [col for col in area.columns if 'province' in col]
province = area[['updateTime'] + province_cols].drop_duplicates()
province.head()

,updateTime,provinceName,provinceEnglishName,province_zipCode,province_confirmedCount,province_suspectedCount,province_curedCount,province_deadCount
0,2020-02-17 13:15:41.559,广东省,Guangdong,440000,1322,0,484,4
21,2020-02-17 13:13:39.175,广东省,Guangdong,440000,1322,0,483,4
42,2020-02-17 13:12:37.938,四川省,Sichuan,510000,495,0,137,3
64,2020-02-17 13:07:31.627,安徽省,Anhui,340000,973,0,278,6
80,2020-02-17 13:07:31.627,河北省,Hebei,130000,301,0,111,3


In [4]:
def calcu_update_freq(var):
    df = province.groupby(['provinceName', var])['updateTime'].last().reset_index()
    df.sort_values(['provinceName', 'updateTime'], inplace=True)
    df['shiftedUpdateTime'] = df.groupby('provinceName').updateTime.shift(1)
    df['updateFreqHours'] = ((df.updateTime - df['shiftedUpdateTime']).dt.total_seconds() / 3600).round(2)
    df['updateValues'] = df[var] - df.groupby('provinceName')[var].shift(1)
    df = df.groupby('provinceName')['updateFreqHours', 'updateValues'].quantile(0.5)
    return df

In [5]:
confirmed_freq = calcu_update_freq('province_confirmedCount')
confirmed_freq.columns = [col+'_confirmed' for col in confirmed_freq.columns]

cured_freq = calcu_update_freq('province_curedCount')
cured_freq.columns = [col+'_cured' for col in cured_freq.columns]

dead_freq = calcu_update_freq('province_deadCount')
dead_freq.columns = [col+'_dead' for col in dead_freq.columns]

update_freq = cured_freq.join(confirmed_freq).join(dead_freq)

In [6]:
update_freq['current_confirmed'] = province.groupby('provinceName')['province_confirmedCount'].max().sort_values()

In [7]:
update_freq.to_csv('update_freq.csv')